In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam,SGD
import numpy as np
import os

# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 2 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# create a data generator
datagenForTrain = ImageDataGenerator(        
    rescale=1./255,
    zoom_range=0.5,
    validation_split=.2)

datagenForValid = ImageDataGenerator(validation_split=.2)


# load and iterate training dataset
train_it = datagenForTrain.flow_from_directory('../data/kaggle_dogcat/train', class_mode='binary', batch_size=batch_size, subset='training')


# load and iterate validation dataset
#valid_it = datagen.flow_from_directory('../data/kaggle_dogcat/train', class_mode='binary', batch_size=batch_size, subset='validation')
valid_it = datagenForValid.flow_from_directory('../data/kaggle_dogcat/train', class_mode='binary', batch_size=batch_size, subset='validation')

# confirm the iterator works
batchX = train_it.next()
batchY = valid_it.next()

# load and iterate test dataset
# test_it = datagen.flow_from_directory('../data/kaggle_dogcat/test', class_mode='binary', batch_size=64)

Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [4]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=batchX[0][0].shape))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(1))

model.add(Activation('softmax'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)  

In [ ]:
#model.fit_generator(train_it, steps_per_epoch=16, validation_data=valid_it, validation_steps=8)

model.fit_generator(
    train_it,
    steps_per_epoch = train_it.samples // batch_size,
    validation_data = valid_it, 
    validation_steps = valid_it.samples // batch_size,
    epochs = epochs)

# history = model.fit_generator(datagen,
#                     steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))

#score = model.evaluate(x_test, y_test, verbose=0)



Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100/100 [==============================] - 381s 4s/step - loss: 7.9662 - acc: 0.5003 - val_loss: 7.8915 - val_acc: 0.5050
Epoch 2/10
100/100 [==============================] - 466s 5s/step - loss: 8.0011 - acc: 0.4981 - val_loss: 8.0708 - val_acc: 0.4938
Epoch 3/10
100/100 [==============================] - 589s 6s/step - loss: 7.9662 - acc: 0.5003 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
100/100 [==============================] - 588s 6s/step - loss: 7.9612 - acc: 0.5006 - val_loss: 7.9114 - val_acc: 0.5038
Epoch 5/10
100/100 [==============================] - 462s 5s/step - loss: 7.9612 - acc: 0.5006 - val_loss: 7.9911 - val_acc: 0.4988
Epoch 6/10
 53/100 [==============>...............] - ETA: 2:49 - loss: 7.9712 - acc: 0.5000

In [ ]:
score = model.evaluate_generator(valid_it,steps=valid_it.samples // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])